In [99]:
import math
from typing import Tuple, List
import time
import pandas as pd
import numpy as np

def gen_ranges(
    COLUMN_LENGTH,
    COLUMN_WIDTH,
    BEARINGPRESSURE,
    LOAD_DEAD,
    LOAD_LIVE,
):
    load_sum = LOAD_DEAD + LOAD_LIVE
    column_diff = COLUMN_LENGTH - COLUMN_WIDTH
    common_term = math.sqrt(BEARINGPRESSURE * (BEARINGPRESSURE * column_diff**2 + 4 * load_sum))

    START_WIDTH = (COLUMN_WIDTH - COLUMN_LENGTH) / 2 + common_term / (2 * BEARINGPRESSURE)
    START_LENGTH = (COLUMN_LENGTH - COLUMN_WIDTH) / 2 + common_term / (2 * BEARINGPRESSURE)

    FTG_LENGTH_MIN = math.ceil(START_LENGTH / 0.05) * 0.05
    FTG_WIDTH_MIN = math.ceil(START_WIDTH / 0.05) * 0.05
    FTG_LENGTH_MAX = FTG_LENGTH_MIN + 0.5
    FTG_WIDTH_MAX = FTG_WIDTH_MIN + 0.5

    FTG_LENGTHS = pd.Series(np.arange(FTG_LENGTH_MIN, FTG_LENGTH_MAX + 0.051, 0.05)).round(2)
    FTG_WIDTHS = pd.Series(np.arange(FTG_WIDTH_MIN, FTG_WIDTH_MAX + 0.051, 0.05)).round(2)

    column_sum = COLUMN_LENGTH + COLUMN_WIDTH
    FTG_DPTH_MIN = (
        math.ceil(
            (
                -column_sum / 4
                + 0.05
                * math.sqrt(
                    25 * column_sum**2
                    + 3 / 238 * math.sqrt(5) * (4 * LOAD_DEAD + 5 * LOAD_LIVE)
                )
            )
            / 0.05
        )
        * 0.05
    )

    FTG_DPTH_MAX = FTG_DPTH_MIN + 0.5
    FTG_DPTHS = pd.Series(np.arange(FTG_DPTH_MIN, FTG_DPTH_MAX + 0.051, 0.05)).round(2)

    FTG_CONC_STRENGTHS = [25, 32, 40, 50, 65]
    FTG_REO_SIZES = [0.012, 0.016, 0.02, 0.024, 0.028, 0.032, 0.036, 0.04]
    FTG_REO_CTS = sorted(set([round(x, 3) for x in range(100, 301, 25)] + [round(x, 3) for x in range(100, 301, 20)]))
    FTG_REO_CTS = [x / 1000 for x in FTG_REO_CTS]

    return (
        FTG_LENGTHS,
        FTG_WIDTHS,
        FTG_DPTHS,
        FTG_CONC_STRENGTHS,
        FTG_REO_SIZES,
        FTG_REO_CTS
    )

def gen_FDN_sizes(COL_LENGTH, COL_WIDTH, ftg_lengths, ftg_widths, ftg_depths):
    sizediff = 0.15
    index = pd.MultiIndex.from_product([ftg_lengths, ftg_widths, ftg_depths], names=['L', 'W', 'D'])
    df = pd.DataFrame(index=index).reset_index()
    df_valid = df[
        (df['L'] == df['W']) |
        ((df['L'] > df['W'] + sizediff) & (COL_LENGTH >= COL_WIDTH)) |
        ((df['W'] > df['L'] + sizediff) & (COL_LENGTH <= COL_WIDTH))
    ]
    return df_valid

def filter_FDN_sizes(combinations, LOAD_DEAD, LOAD_LIVE, BEARINGPRESSURE):
    return combinations[
        (LOAD_DEAD + LOAD_LIVE + 6 * combinations['D'] * combinations['L'] * combinations['W']) < 
        BEARINGPRESSURE * combinations['L'] * combinations['W']
    ]

def gen_REO_sizes(reo_sizes,reo_cts):
    index = pd.MultiIndex.from_product([reo_sizes, reo_cts], names=['size', 'cts'])
    df = pd.DataFrame(index=index).reset_index()
    return df

def filter_REO_sizes(ftg_combinations, reo_combinations, LOAD_DEAD, LOAD_LIVE, FTG_COVER, COL_LENGTH, COL_WIDTH):
    dfL = ftg_combinations.merge(reo_combinations, how='cross')
    dsL = 2*dfL['D']-dfL['size']-2*FTG_COVER
    AstminL = 456*dfL['D']**2*np.sqrt(dfL['fc'])/dsL
    AstshrL = 3*(7*COL_LENGTH-10*dfL['L'])**2/1600*(4*LOAD_DEAD+5*LOAD_LIVE+6*dfL['D']*dfL['L']*dfL['W'])/(dsL*dfL['W']*dfL['L'])
    minreoL = np.maximum(AstminL,AstshrL)
    dfL = dfL[minreoL <= (785398.16339745 * dfL['size']**2) / dfL['cts']]
    dfL.drop_duplicates(inplace=True)

   
    # dsW = 2*dfW['D']-dfW['size']-2*0.04-2*FTG_COVER
    # AstminW = 456*dfW['D']**2*np.sqrt(dfW['fc'])/dsW
    # AstshrW = 3*(7*COL_WIDTH-10*dfW['W'])**2/1600*(4*LOAD_DEAD+5*LOAD_LIVE+6*dfW['D']*dfW['L']*dfW['W'])/(dsW*dfW['W']*dfW['L'])
    # minreoW = np.maximum(AstminW,AstshrW)
    # dfW = dfW[minreoW <= dfW['area_per_m']]



def filter_REO_sizes_2(ftg_combinations, reo_combinations, FTG_COVER):
    df = pd.DataFrame({'D': ftg_combinations['D'], 'fc': ftg_combinations['fc']})
    df.drop_duplicates(inplace=True)
    df = df.merge(reo_combinations, how='cross')
    df = df.merge(reo_combinations, how='cross', suffixes=('_L', '_W'))
    df.drop_duplicates(inplace=True)
    minL = 456*df['D']**2*np.sqrt(df['fc'])/(2*df['D']-df['size_L']-2*FTG_COVER)
    minW = 456*df['D']**2*np.sqrt(df['fc'])/(2*df['D']-2*df['size_L']-df['size_W']-2*FTG_COVER)
    df = df[(minL <= (785398.16339745 * df['size_L']**2) / df['cts_L']) & (minW <= (785398.16339745 * df['size_W']**2) / df['cts_W'])]
    return df
# Example usage
if __name__ == "__main__":
    start_time = time.time()

    COL_LENGTH: float = 0.9
    COL_WIDTH: float = 0.9
    FTG_COVER: float = 0.060
    BEARINGPRESSURE: float = 600
    LOAD_DEAD: float = 1000
    LOAD_LIVE: float = 300

    # Generate arrays
    size_ranges = gen_ranges(
        COL_LENGTH, COL_WIDTH, BEARINGPRESSURE, LOAD_DEAD, LOAD_LIVE
    )

    # Generate valid combinations
    FDN_sizes_all = gen_FDN_sizes(COL_LENGTH, COL_WIDTH, size_ranges[0], size_ranges[1], size_ranges[2])

    FDN_sizes_valid = filter_FDN_sizes(FDN_sizes_all, LOAD_DEAD, LOAD_LIVE, BEARINGPRESSURE)

    # Create a DataFrame with all combinations of valid sizes and concrete strengths
    fc_range = pd.Series(size_ranges[3])
    combined_df = FDN_sizes_valid.assign(key=1).merge(fc_range.to_frame('fc').assign(key=1), on='key').drop('key', axis=1)

    reo_sizes_all = gen_REO_sizes(size_ranges[4], size_ranges[5])

    reo_sizes_filtered = filter_REO_sizes(combined_df, reo_sizes_all, LOAD_DEAD, LOAD_LIVE, FTG_COVER, COL_LENGTH, COL_WIDTH)

    test = filter_REO_sizes_2(combined_df, reo_sizes_all, FTG_COVER)
    print(test)


    # print(f"FDN_sizes_all shape:\t{FDN_sizes_all.shape}")
    # print(f"FDN_sizes_valid shape:\t{FDN_sizes_valid.shape}")
    # print(f"combined_df shape:\t{combined_df.shape}")
    # print(combined_df)
    
    end_time = time.time()
    execution_time = end_time - start_time
    print(f"\nExecution time: {execution_time:.2f} seconds")

            D  fc  size_L  cts_L  size_W  cts_W
0        0.35  25   0.012    0.1   0.012  0.100
1        0.35  25   0.012    0.1   0.012  0.120
2        0.35  25   0.012    0.1   0.012  0.125
3        0.35  25   0.012    0.1   0.012  0.140
4        0.35  25   0.012    0.1   0.012  0.150
...       ...  ..     ...    ...     ...    ...
1109755  0.90  65   0.040    0.3   0.040  0.250
1109756  0.90  65   0.040    0.3   0.040  0.260
1109757  0.90  65   0.040    0.3   0.040  0.275
1109758  0.90  65   0.040    0.3   0.040  0.280
1109759  0.90  65   0.040    0.3   0.040  0.300

[737318 rows x 6 columns]

Execution time: 0.52 seconds
